In [2]:
# ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in 
# emails, as well as in banner ads on Facebook, Twitter, and Google. 
# They want to know how the two ads are performing on each of the different platforms on each day of the week.

In [3]:
import pandas as pd

In [7]:
# load the dataframe CSV
ad_clicks = pd.read_csv('Micro project in Python- AB Testing with Pandads_Dataset.csv')
print(ad_clicks.head(3))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  


In [9]:
# How many views came from each platfrom?
visit_platfrom = ad_clicks.groupby('utm_source').user_id.count().reset_index()
# Rename the columns to be more representative
visit_platfrom.columns = ['platfrom', 'nr_of_views']
print(visit_platfrom)

   platfrom  nr_of_views
0     email          255
1  facebook          504
2    google          680
3   twitter          215


In [12]:
# Which platfrom drived the most visit?
max_view = visit_platfrom[visit_platfrom.nr_of_views == visit_platfrom.nr_of_views.max()]
print(max_view)

  platfrom  nr_of_views
2   google          680
